In [171]:
import numpy as np
import pandas as pd

In [172]:
#One-hot encoding the tendecy variable(3 categories). and dropping DR 
raw_data=pd.read_csv("../data/cleaned_data.csv")
tendecy_one_hot=pd.get_dummies(raw_data.loc[:,"Tendency"])
raw_data=pd.concat([raw_data,tendecy_one_hot],axis=1)
raw_data.drop(['Tendency','DR'],axis=1, inplace=True)
raw_data.columns

Index(['FileName', 'Date', 'SegFile', 'b', 'e', 'LBE', 'LB', 'AC', 'FM', 'UC',
       'ASTV', 'MSTV', 'ALTV', 'MLTV', 'DL', 'DS', 'DP', 'Width', 'Min', 'Max',
       'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance', 'NSP',
       'left_assymetric', 'right_symmetric', 'symmetric'],
      dtype='object')

In [173]:
#Normalizing only the continuous variables
continuous_attributes=raw_data.iloc[:,6:26]
categorical_attributes=raw_data.iloc[:,26:]
continuous_attributes=(continuous_attributes-continuous_attributes.min())/(continuous_attributes.max()-continuous_attributes.min())
dataset=pd.concat([continuous_attributes,categorical_attributes.iloc[:,1:],categorical_attributes.iloc[:,0]],axis=1)
dataset.to_csv("../data/cleaned_normalized_data.csv",index=False)

In [174]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import math

In [175]:
#dividing into train and test data
dataset_array=dataset.values
no_of_instances=len(dataset_array)
split=math.floor(0.75*no_of_instances)
data_x=dataset_array[:,:-1]
data_y=dataset_array[:,-1]
X_train=dataset_array[:split,:-1]
y_train=dataset_array[:split,-1]
X_test=dataset_array[split:,:-1]
y_test=dataset_array[split:,-1]

In [181]:
#Define the SVC classifier using sklearn package
classifier = svm.SVC(kernel='linear', C = 1.0, verbose=True)

#Training the model
classifier.fit(X_train,y_train)

#testing the model
y_pred=classifier.predict(X_test)

#getting the classification metrics
confusion_mtx = confusion_matrix(y_test, y_pred) 
print("\nConfusion Matrix=\n",confusion_mtx)
target_names = ['normal', 'suspect', 'pathological']
print(classification_report(y_test, y_pred, target_names=target_names))
print("accuracy= ",(confusion_mtx.diagonal().sum()/confusion_mtx.sum())*100)



[LibSVM]
Confusion Matrix=
 [[293 134   9]
 [  4   5   0]
 [  4  10  73]]
              precision    recall  f1-score   support

      normal       0.97      0.67      0.80       436
     suspect       0.03      0.56      0.06         9
pathological       0.89      0.84      0.86        87

 avg / total       0.94      0.70      0.79       532

accuracy=  69.7368421053
